# Programming Assignment: 
## Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [3]:
import json

In [4]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [5]:
print recipes[0]

{u'cuisine': u'greek', u'id': 10259, u'ingredients': [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']}


### Составление корпуса

In [6]:
#from six.moves import xrange, zip as izip
from gensim import corpora, models
import numpy as np

C:\Users\kazantseva\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
No handlers could be found for logger "gensim.models.doc2vec"


Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [7]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [8]:
print texts[0]
print corpus[0]

[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [9]:
np.random.seed(76543)
# здесь код для построения модели:
lda = models.LdaModel(corpus, num_topics = 40, passes = 5)
tops = lda.show_topics(num_topics=40, num_words = 10)

In [12]:
import collections as coll
import re
import numpy as np
words = {}
for top in tops:
    s = top[1]
    tokens = re.findall('"(\d+)"', s)
    real_words = []
    for token in tokens:
        real_words.append(dictionary[int(token)])
    words[top[0]] = real_words
c = coll.Counter(np.hstack((words.values())))
#print np.hstack((words.values()))

In [13]:
ws = []
for w in ["salt", "sugar", "water", "mushrooms", "chicken", "eggs"]:
    ws.append(c[w])
print ws

[20, 7, 10, 1, 1, 2]


In [14]:
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))
#save_answers1(*ws)

### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [15]:
import copy
dictionary2 = copy.deepcopy(dictionary)

__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [16]:
dict_sort = filter(lambda x : x[1] > 4000, dictionary2.dfs.items())
ingr_list = [k[0] for k in dict_sort]
print dict(dict_sort)
print ingr_list
dict_size_before = len(dictionary2)
print 'dict_size_before', dict_size_before
dictionary2.filter_tokens(bad_ids = ingr_list)
dict_size_after = len(dictionary2)
print 'dict_size_after', dict_size_after

{0: 4438, 4: 7380, 9: 4385, 12: 4784, 45: 7972, 48: 6434, 17: 18048, 21: 4847, 54: 7971, 100: 6236, 29: 7457, 117: 4632}
[0, 4, 9, 12, 17, 21, 29, 45, 48, 54, 100, 117]
dict_size_before 6714
dict_size_after 6702


In [17]:
corpus_size_before = sum([len(ingrs) for ingrs in corpus])
corpus2 = [dictionary2.doc2bow(text) for text in texts]
corpus_size_after = sum([len(ingrs) for ingrs in corpus2])
print 'corpus_size_before', corpus_size_before
print 'corpus_size_after', corpus_size_after

corpus_size_before 428249
corpus_size_after 343665


In [18]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

In [19]:
#save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after)

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [20]:
np.random.seed(76543)
# здесь код для построения модели:
lda2 = models.LdaModel(corpus2, num_topics = 40, passes = 5)
coher2 = lda2.top_topics(corpus2)
coher = lda.top_topics(corpus)

TypeError: top_topics() got an unexpected keyword argument 'dictionary'

In [21]:
coher2 = lda2.top_topics(corpus2)
coher = lda.top_topics(corpus)
mean_coher = np.mean([c[1] for c in coher])
mean_coher2 = np.mean([c[1] for c in coher2])

In [23]:
print mean_coher
print mean_coher2
#print [c[1] for c in coher2]

-699.4559739709771
-737.7190246856348


In [24]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

In [25]:
save_answers3(mean_coher, mean_coher2)

Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

In [153]:
doc_top = lda2.get_document_topics(corpus2[0])
print doc_top
str_top = []
for t in doc_top:
    str_top.append((t[0], lda2.get_topic_terms(t[0])))
print str_top
print corpus2[0]
#print texts[0]
#print dictionary2[1880], dictionary2[2837], dictionary2[3225]

[(5, 0.1307525644192953), (7, 0.505189963958857), (11, 0.2484324716218466)]
[(5, [(2334, 0.08536074755505069), (5447, 0.06464513745907814), (1837, 0.05085421394385921), (4015, 0.043583587940220776), (5710, 0.03447378413746958), (1607, 0.03445996556144133), (1874, 0.029179262440199423), (3102, 0.026626401811099248), (3209, 0.02614563450692993), (3746, 0.02358699386249915)]), (7, [(5442, 0.1228916843733982), (501, 0.11072366617875934), (5758, 0.04144982542265059), (5144, 0.0410032035616342), (5099, 0.03964806115181897), (5459, 0.035216055572896095), (5829, 0.030780122572984552), (3452, 0.027942943011875852), (6163, 0.027541274735641903), (6584, 0.025100535613163975)]), (11, [(643, 0.1287057730700716), (5833, 0.05376265967118062), (6110, 0.05309558274634057), (1533, 0.05047803355438952), (6584, 0.04862422265220821), (2400, 0.03727602897003153), (1612, 0.03565613815578912), (2709, 0.029935878543843875), (4330, 0.027267545982844702), (1196, 0.026763342202766695)])]
[(1880, 1), (2837, 1), (3

Также выведите содержимое переменной *.alpha* второй модели:

In [145]:
print lda2.alpha

[0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025]


У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [154]:
np.random.seed(76543)
# здесь код для построения модели:
lda3 = models.LdaModel(corpus2, num_topics = 40, passes = 5, alpha = 1)
#coher2 = lda2.top_topics(corpus = corpus2, dictionary = dictionary2)
#coher = lda.top_topics(corpus = corpus, dictionary = dictionary)

In [159]:
doc_top3 = lda3.get_document_topics(corpus2[0])
print doc_top3

[(0, 0.02127962739865838), (1, 0.021276595744681655), (2, 0.021286176786625182), (3, 0.02200828246520371), (4, 0.021276595744680847), (5, 0.021302903292659565), (6, 0.02155284710875066), (7, 0.024303029881803603), (8, 0.021284575268651344), (9, 0.02136918822887798), (10, 0.021895873714714742), (11, 0.02131016992701198), (12, 0.021303198099398884), (13, 0.021276595744680847), (14, 0.02127659574468801), (15, 0.027078629319985545), (16, 0.021594247758819605), (17, 0.042603188696847503), (18, 0.021444072792535025), (19, 0.07677655555453923), (20, 0.021312700402165697), (21, 0.021276728382191495), (22, 0.021318480205658488), (23, 0.02131455600043065), (24, 0.021276595744680847), (25, 0.0448489435827903), (26, 0.021303487125494763), (27, 0.021352021007044555), (28, 0.04308216678417664), (29, 0.03493951912271069), (30, 0.021278289220592897), (31, 0.021276595744693216), (32, 0.022056464687257516), (33, 0.021382179000607446), (34, 0.02137677101866378), (35, 0.02178102340454862), (36, 0.02138823

In [160]:
count_model3 = sum([len(lda3.get_document_topics(t, minimum_probability=0.01)) for t in corpus2])
count_model2 = sum([len(lda2.get_document_topics(t, minimum_probability=0.01)) for t in corpus2])
print count_model2
print count_model3

198089
1590960


In [162]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

In [163]:
save_answers4(count_model2, count_model3)

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [172]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

C:\Users\kazantseva\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [198]:
y = [recipe["cuisine"] for recipe in recipes] 
count_topics = lda2.get_topics().shape[0]
docs_tops = [lda2.get_document_topics(c2) for c2 in corpus2]
docs_tops_final = [[dict(doc)[col] if col in dict(doc).keys() else 0 for col in range(count_topics)] for doc in docs_tops]
rf_model = RandomForestClassifier(n_estimators = 100)
sc = cross_val_score(rf_model, docs_tops_final, y, cv = 3).mean()

In [199]:
#print len(y)
#print dict(docs_tops[0])
#print docs_tops_final[:10]
print sc

0.5579029714372511


In [200]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

In [201]:
save_answers5(sc)

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [ ]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(t, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print w

### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [ ]:
import pandas
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [ ]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [ ]:
# Визуализируйте матрицу


Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 